In [1]:
import os
import sys
import time

from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

from src import utils
from src import bilstm
import src.dataset as dset
import src.pytorch_utils as ptu
import src.chu_liu_edmonds as chu

import warnings
warnings.filterwarnings('ignore')

seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

np.random.seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
models_path = 'models'

cuda


In [2]:
train_dataset = dset.DataSet('data/train.labeled', tqdm_bar=True)
# train_dataset = dset.DataSet('data/test.labeled', tqdm_bar=True)
test_dataset = dset.DataSet('data/test.labeled', train_dataset=train_dataset, tqdm_bar=True)
# comp_dataset = dset.DataSet('data/comp.unlabeled', train_dataset=train_dataset, tagged=False, tqdm_bar=True)

100%|██████████| 25325/25325 [00:03<00:00, 6711.31it/s]


In [3]:
version = 'V1_2.0'
save = True

model = bilstm.BiLSTM(train_dataset=train_dataset,
                      word_embed_dim=100,
                      tag_embed_dim=25,
                      hidden_dim=125,
                      num_layers=2,
                      bias=True,
                      mlp1_dim=100,
                      mlp2_dim=1,
                      p_dropout=0.1,
                      word_dropout=0.25)

checkpoint = ptu.Checkpoint(models_path=models_path,
                            version=version,
                            model=model,
                            score=lambda y_true, y_pred: (np.array(y_true) == np.array(y_pred)).mean(),
                            loss_decision_func=utils.loss_decision_func,
                            out_decision_func=lambda y_pred, flat_y_pred, mask, padding: flat_y_pred.argmax(axis=1),
#                             out_decision_func=test_chu_liu_edmonds,
                            seed=42,
                            optimizer=torch.optim.Adam,
                            criterion=nn.NLLLoss,
                            save=save,
                            prints=True,
                           )

model version: V1_2.0
Number of parameters 2097001 trainable 2097001


In [ ]:
hyperparam_list = [
    {'train_epochs': 5, 'batch_size': 16, 'optimizer_params': {'lr': 1e-3, }, 'lr_decay': 0.0},
    {'train_epochs': 95, 'batch_size': 64, 'optimizer_params': {'lr': 4e-4}, 'lr_decay': 0.0},
#     {'train_epochs': 5, 'batch_size': 64, 'optimizer_params': {'lr': 1e-4}, 'lr_decay': 0.0},
#     {'train_epochs': 35, 'batch_size': 64, 'optimizer_params': {'lr': 1e-5}, 'lr_decay': 0.0},
#     {'train_epochs': 5, 'batch_size': 128, 'optimizer_params': {}, 'lr_decay': 0.07},
#     {'train_epochs': 50, 'batch_size': 32, 'optimizer_params': {'lr': 4e-4}, 'lr_decay': 0.07},
#     {'train_epochs': 50, 'batch_size': 2048, 'optimizer_params': {}, 'lr_decay': 0.07},
#     {'train_epochs': 5, 'batch_size': 1024, 'optimizer_params': {'weight_decay': 1e-6}, 'lr_decay': 0.07},
#     {'train_epochs': 50, 'batch_size': 2048, 'optimizer_params': {'weight_decay': 1e-6}, 'lr_decay': 0.07},
#     {'train_epochs': 50, 'batch_size': 5000, 'optimizer_params': {'weight_decay': 0}, 'lr_decay': 0.05},
]

for session in hyperparam_list:
    checkpoint.train(device=device,
                     train_dataset=train_dataset.dataset,
                     val_dataset=test_dataset.dataset,
                     prints=True,
                     epochs_save=5,
                     save=save,
                     **session)

epoch   1/  5 | train_loss 2.19134 | val_loss 2.18516 | train_score 0.30509 | val_score 0.29524 | train_time   0.42 min
epoch   2/  5 | train_loss 1.60240 | val_loss 1.60150 | train_score 0.53955 | val_score 0.54721 | train_time   0.85 min
epoch   3/  5 | train_loss 1.36056 | val_loss 1.37565 | train_score 0.60301 | val_score 0.60399 | train_time   1.28 min
epoch   4/  5 | train_loss 1.26888 | val_loss 1.30515 | train_score 0.61326 | val_score 0.60703 | train_time   1.71 min
epoch   5/  5 | train_loss 1.21746 | val_loss 1.27309 | train_score 0.61757 | val_score 0.61137 | train_time   2.15 min
epoch   6/100 | train_loss 1.18319 | val_loss 1.24164 | train_score 0.61982 | val_score 0.60885 | train_time   2.53 min
epoch   7/100 | train_loss 1.17536 | val_loss 1.23860 | train_score 0.62053 | val_score 0.60944 | train_time   2.92 min
epoch   8/100 | train_loss 1.16951 | val_loss 1.23675 | train_score 0.62042 | val_score 0.60873 | train_time   3.30 min
epoch   9/100 | train_loss 1.16316 | val

In [ ]:
# load = ptu.load_model(version=version, models_path=models_path, epoch=-1, seed=42)
# loss, score, y_pred, y_true = checkpoint.predict(test_dataset.dataset,
#                                                  batch_size=32,
#                                                  device=device,
#                                                  results=True,
#                                                  decision_func=chu.test_chu_liu_edmonds)
# print(score)
# print(loss)

In [ ]:
# %%time
# checkpoint.model = checkpoint.model.to(device)
# checkpoint.model.train()
# batch_size = 1

# # loss_sum = np.array([])
# # y_pred = np.array([])
# # y_true = np.array([])

# loader = torch.utils.data.DataLoader(dataset=train_dataset.dataset, batch_size=batch_size, shuffle=True)
# for batch in loader:
# #     words, tags, lens, y = batch
# #     out = checkpoint.model.forward(words.to(device), tags.to(device), lens, prints=True)
#     loss, flat_y, flat_out, mask, out, y = utils.loss_decision_func(checkpoint, device, batch, prints=True)

# #     loss_sum = np.append(loss_sum, float(loss.data))
    
# #     y_pred = np.append(y_pred, checkpoint.out_decision_func(out.detach().cpu().numpy()))
    
# #     y_true = np.append(y_true, batch[-1].detach().cpu().numpy())
#     break
# torch.cuda.empty_cache()

In [ ]:
# import hyperopt as hpo

# def Counter():
#     for i in range(999999999999):
#         yield i

# init_space = {
    
# }

# session_space = {
    
# }

# def init_objective(space, save=False):
#     score = 0.0
#     return - score

# def session_objective(space, save=False):
#     score = 0.0
#     return - score

# _ = hpo.fmin()